In [2]:
import tensorflow as tf

from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Conv2D,
    BatchNormalization,
    MaxPooling2D,
    Dropout,
    GlobalAveragePooling2D,
)

from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist, cifar10
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
print(tf.__version__)
print(tf.config.list_physical_devices("GPU"))

2.17.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.reshape(-1, 32, 32, 3).astype("float32")
x_test = x_test.reshape(-1, 32, 32, 3).astype("float32")


# Add a channel dimension

In [5]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

In [6]:
class CNNblock(tf.keras.layers.Layer):

    def __init__(self, filters, kernel_size=3):
        super(CNNblock, self).__init__()
        self.conv = Conv2D(
            filters,
            kernel_size,
            input_shape=(32, 32, 3),
            activation="relu",
            padding="same",
        )
        self.norm = BatchNormalization()

    def __call__(self, inputs, training=False):
        x = self.conv(inputs, training=training)
        x = self.norm(x, training=training)
        return x

In [7]:
class ResNet_Block(tf.keras.layers.Layer):
    def __init__(self, channels):
        super(ResNet_Block, self).__init__()
        self.cnn1 = CNNblock(channels[0])
        self.cnn2 = CNNblock(channels[1])
        self.cnn3 = CNNblock(channels[2])
        self.pool = MaxPooling2D()
        self.identity_mapping = Conv2D(channels[1], 1, padding="same")

    def call(self, input, training=False):
        x = self.cnn1(input, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input), training=training)
        x = self.pool(x)
        return x

In [8]:
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.block1 = ResNet_Block([32, 32, 64])
        self.block2 = ResNet_Block([64, 64, 128])
        self.block3 = ResNet_Block([128, 128, 256])
        self.block4 = ResNet_Block([256, 256, 512])

        self.pool = GlobalAveragePooling2D()
        self.layer1 = Dense(512, activation="relu")
        self.dropout = Dropout(0.5)
        self.layer2 = Dense(128, activation="softmax")
        self.classifier = Dense(num_classes, activation="softmax")

    def call(self, input, training=False):
        x = self.block1(input, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.block4(x, training=training)
        x = self.pool(x)
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.classifier(x)
        return x

    def model(self):
        x = tf.keras.layers.Input(shape=(32, 32, 3))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [10]:
model = ResNet(10)

In [11]:
model.model().summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net__block_4 (ResNet_Block) │ (None, 16, 16, 64)     │        29,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net__block_5 (ResNet_Block) │ (None, 8, 8, 128)      │       152,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net__block_6 (ResNet_Block) │ (None, 4, 4, 256)      │       608,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net__block_7 (ResNet_Block) │ (None, 2, 2, 512)      │     2,430,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,489,066 (13.31 MB)

 Trainable params: 3,485,226 (13.30 MB)

 Non-trainable params: 3,840 (15.00 KB)

In [13]:
model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(x_train, y_train, batch_size=64, epochs=20)

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 22s 20ms/step - accuracy: 0.6786 - loss: 0.9377
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.7699 - loss: 0.6781
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.8167 - loss: 0.5388
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.8540 - loss: 0.4266
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.8828 - loss: 0.3420
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9059 - loss: 0.2822
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9233 - loss: 0.2259
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9370 - loss: 0.1861
Epoch 9/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9469 - loss: 0.1605
Epoch 10/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9541 - loss: 0.1362
Epoch 11/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.9599 - loss: 0.1204
Epoch 12/20
782/782 ━━━━━━━━━━━━━━━━━━━━

In [14]:
model.evaluate(x_test, y_test, batch_size=64)

157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.8282 - loss: 0.9754


[0.9878829121589661, 0.8274000287055969]

In [34]:
import matplotlib.pyplot as plt

erorr = 0
for i in range(100):
    # plt.imshow(x_test[i] / 255.0)
    # plt.title(y_test[i])
    # plt.show()
    image = x_test[i]
    y_hat = y_test[i]

    classes = {
        0: "airplane",
        1: "automobile",
        2: "bird",
        3: "cat",
        4: "deer",
        5: "dog",
        6: "frog",
        7: "horse",
        8: "ship",
        9: "truck",
    }
    predicted = model.predict(image.reshape(1, 32, 32, 3))

    if classes[predicted.argmax()] != classes[y_hat[0]]:
        erorr += 1
    print(classes[y_hat[0]], classes[predicted.argmax()])

print(f"in the first 100 images the model made {erorr} errors")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
cat automobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
ship ship
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
ship truck
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
airplane airplane
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
frog frog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
frog frog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
automobile automobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
frog deer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
cat cat
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
automobile automobile
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
airplane airplane
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
truck truck
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
dog deer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
horse horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
truck truck
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
ship airplane
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
dog dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
horse horse
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
ship ship
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/st